In [1]:
import torch

from diffusion_3d.chestct.autoencoder.vae_gan.config import get_config
from diffusion_3d.chestct.autoencoder.vae_gan.model import AdaptiveVAEGAN
from diffusion_3d.datasets.ct_rate import CTRATEDataModule
from diffusion_3d.utils.visualize import plot_scans

In [2]:
config = get_config()
config

Munch({'data': Munch({'csvpath': '/raid3/arjun/ct_pretraining/csvs/sources.csv', 'datapath': '/raid3/arjun/ct_pretraining/scans/', 'checkpointspath': '/raid3/arjun/checkpoints/adaptive_autoencoder/', 'limited_dataset_size': None, 'allowed_spacings': ((0.4, 7), (-1, -1), (-1, -1)), 'allowed_shapes': ((225, 256), (256, -1), (256, -1)), 'train_augmentations': [Munch({'__fn_name__': 'random_resize_2d', 'min_shape': (None, 256, 256), 'max_shape': (None, 256, 256), 'interpolation_mode': 'bilinear'}), Munch({'__fn_name__': 'random_crop', 'target_shape': (-1, 256, 256)}), Munch({'__fn_name__': 'pad_to_multiple_of', 'min_size': (32, 256, 256), 'tile_size': (32, 32, 32)}), Munch({'__fn_name__': 'random_rotate', 'degrees': 15}), Munch({'__fn_name__': 'random_windowing', 'window_choices': [(3000, 500)], 'normalize_range': (-1.0, 1.0)}), Munch({'__fn_name__': 'random_horizontal_flip', 'probability': 0.5}), [[0.4, 0.3, 0.3], [], [Munch({'__fn_name__': 'random_gaussian_blurring', 'sigma_range': (0, 1

In [3]:
datamodule = CTRATEDataModule(config.data)
dataloader = datamodule.val_dataloader()
len(dataloader)

valid:   0%|          | 0/2579 [00:00<?, ?it/s]

No. of valid datapoints: 648


216

In [4]:
checkpoint_path = r'/raid3/arjun/checkpoints/adaptive_autoencoder/v9__2025_02_20/version_0/checkpoints/last.ckpt'

model = AdaptiveVAEGAN.load_from_checkpoint(checkpoint_path, map_location='cpu')
model.eval()

/home/ubuntu/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/ubuntu/miniconda3/lib/python3.10/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.


AdaptiveVAEGAN(
  (encoder): SwinV23DModel(
    (embeddings): SwinV23DEmbeddings(
      (patch_embeddings): SwinV23DPatchEmbeddings(
        (patch_embeddings): Conv3d(1, 12, kernel_size=(4, 4, 4), stride=(4, 4, 4))
      )
      (layer_norm): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.1, inplace=False)
    (encoder): SwinV23DEncoder(
      (stages): ModuleList(
        (0): SwinV23DStage(
          (blocks): ModuleList(
            (0-1): 2 x SwinV23DBlock(
              (w_layer): SwinV23DLayer(
                (mhsa): SwinV23DMHSA(
                  (W_qkv): Linear(in_features=12, out_features=36, bias=True)
                  (proj): Linear(in_features=12, out_features=12, bias=True)
                  (proj_drop): Dropout(p=0.1, inplace=False)
                  (cpb_mlp): Sequential(
                    (0): Linear(in_features=3, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_fe

In [5]:
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        if i >= 1:
            break

        x = batch['scan'].to(model.device)

        losses = []
        reconstructed = model.process_step(x, losses, 'valid', 0)[1].cpu()
        print(f'Batch idx: {i}')
        print(reconstructed.min(), reconstructed.max())
        print(x.shape, reconstructed.shape)
        print((x.min(), x.max()), (reconstructed.min(), reconstructed.max()))

        display(losses)

        reconstructed[:, 0, :, 0, 0] = -1.
        reconstructed[:, 0, :, -1, -1] = 1.

        for i in range(x.shape[0]):
            plot_scans([x[i][0], reconstructed[i][0]], ['Original', 'Reconstructed'])

torch.cuda.empty_cache()

/home/ubuntu/miniconda3/lib/python3.10/site-packages/lightning/pytorch/core/module.py:445: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


Batch idx: 0
metatensor(-1.) metatensor(1.)
torch.Size([3, 1, 256, 256, 256]) torch.Size([3, 1, 256, 256, 256])
(metatensor(-1.), metatensor(1.)) (metatensor(-1.), metatensor(1.))



[
    {
        'reconstruction_loss': metatensor(0.6316),
        'kl_loss': metatensor(0.),
        'perceptual_loss': metatensor(1.6588),
        'generator_loss': metatensor(0.),
        'autoencoder_loss': metatensor(0.7975),
        'disc_fake_loss': metatensor(0.0688),
        'disc_real_loss': metatensor(0.7427),
        'discriminator_loss': metatensor(0.4057)
    }
]

interactive(children=(IntSlider(value=0, description='z', max=255), Output()), _dom_classes=('widget-interact'…

interactive(children=(IntSlider(value=0, description='z', max=255), Output()), _dom_classes=('widget-interact'…

interactive(children=(IntSlider(value=0, description='z', max=255), Output()), _dom_classes=('widget-interact'…

# Check latent space

In [ ]:
raise Exception("Don't want to automatically run beyond this point")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 raise Exception("Don't want to automatically run beyond this point")                         │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
Exception: Don't want to automatically run beyond this point

In [ ]:
dataloader_iter = iter(dataloader)
batch1 = next(dataloader_iter)
batch2 = next(dataloader_iter)
x1, x2 = batch1['scan'], batch2['scan']
(batch1['uid'], batch2['uid']), (x1.shape, x2.shape)


(
    (
        ['train_17513_a_1', 'train_3173_a_1', 'train_19337_d_1'],
        ['train_19337_a_1', 'train_14119_a_1', 'train_18480_a_1']
    ),
    (torch.Size([3, 1, 256, 256, 256]), torch.Size([3, 1, 256, 256, 256]))
)

In [ ]:
with torch.no_grad():
    _, reconstructed1, _, adapted_encoded1, _, _ = model.process_step(x1, [], 'valid', 0)
    _, reconstructed2, _, adapted_encoded2, _, _ = model.process_step(x2, [], 'valid', 0)

(reconstructed1.shape, reconstructed2.shape), (adapted_encoded1.shape, adapted_encoded2.shape)


(
    (torch.Size([3, 1, 256, 256, 256]), torch.Size([3, 1, 256, 256, 256])),
    (torch.Size([3, 768, 24, 8, 8]), torch.Size([3, 768, 24, 8, 8]))
)

In [ ]:
adapted_encoded_inter = (adapted_encoded1 + adapted_encoded2) / 2
adapted_encoded_inter.shape

torch.Size([3, 768, 24, 8, 8])

In [ ]:
from torch.nn import functional as F

with torch.no_grad():
    decoded_inter = model.decode(adapted_encoded_inter)
    reconstructed_inter = F.interpolate(decoded_inter, x1.shape[2:], mode="trilinear")
reconstructed_inter.shape

torch.Size([3, 1, 256, 256, 256])

In [ ]:
plot_scans([x1[0][0], x2[0][0], reconstructed1[0][0], reconstructed2[0][0], reconstructed_inter[0][0]], ['Scan1', 'Scan2', 'Reconstructed1', 'Reconstructed2', 'Reconstructed interpolated latent'], cols=2)

interactive(children=(IntSlider(value=0, description='z', max=255), Output()), _dom_classes=('widget-interact'…

In [11]:
plot_scans((reconstructed1 - reconstructed2).abs()[0][0])

interactive(children=(IntSlider(value=0, description='z', max=255), Output()), _dom_classes=('widget-interact'…